In [1]:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
#pip install torch-geometric pytorch-lightning --quiet

In [1]:
pip install scikit-image -U

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Implementing Hierarchical Graph Neural Network (HGNN)

In [1]:
from skimage.segmentation import slic
from skimage.util import img_as_float

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
import pytorch_lightning as pl
from skimage.segmentation import slic
from skimage.util import img_as_float
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.loader import DataLoader as GeoDataLoader
import os

In [47]:
class HGNN(pl.LightningModule):
    def __init__(self, num_features, num_classes, hidden_dim=64, num_heads=4):
        super(HGNN, self).__init__()
        self.gat1 = GATConv(num_features, hidden_dim, heads=num_heads)
        self.gat2 = GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.gat3 = GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.fc = nn.Linear(hidden_dim * num_heads, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x, edge_index):
        x = F.relu(self.gat1(x, edge_index))
        x = F.relu(self.gat2(x, edge_index))
        x = F.relu(self.gat3(x, edge_index))
        x = self.readout(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

    def readout(self, x):
        return torch.mean(x, dim=0, keepdim=True)

    def training_step(self, batch, batch_idx):
        x, edge_index, y = batch.x, batch.edge_index, batch.y
        logits = self(x, edge_index)
        loss = self.loss_fn(logits, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Creating Fashion MNIST SuperPixel Dataset

In [89]:
class FashionMNISTSuperpixelDataset(datasets.FashionMNIST):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False, n_segments=100, compactness=10):
        super().__init__(root, train=train, transform=transform, target_transform=target_transform, download=download)
        self.n_segments = n_segments
        self.compactness = compactness

    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        img_np = img.squeeze().numpy()
        img_float = img_as_float(img_np)
        segments = slic(img_float, n_segments=self.n_segments, compactness=self.compactness, start_label=1, channel_axis=None)
        x, edge_index = self.construct_graph(segments, img_float)
        return Data(x=x, edge_index=edge_index, y=target)

    def construct_graph(self, segments, img_float):
        num_segments = np.max(segments) + 1
        x = []
        for i in range(num_segments):
            mask = segments == i
            if np.any(mask):
                features = img_float[mask].mean(axis=0)
                x.append(features)
            else:
                # Handle empty segments by adding a zero vector
                x.append(np.zeros_like(img_float[0]))
        
        # Ensure x is not empty and contains only compatible elements
        if not x:
            raise ValueError("No valid segments found")
        
        # Convert x to a list of numpy arrays
        x = [np.array(features) for features in x]
        
        # Pad x to a fixed size
        max_segments = 100  # Define a fixed size for padding
        padded_x = np.zeros((max_segments, img_float.shape[1]))
        padded_x[:len(x)] = x
        
        # Convert padded_x to a tensor
        x = torch.tensor(padded_x, dtype=torch.float)

        edge_index = []
        for i in range(num_segments):
            for j in range(num_segments):
                if i != j and np.any(segments == i) and np.any(segments == j):
                    edge_index.append([i, j])
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

        return x, edge_index

In [91]:
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir='./data', batch_size=32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def prepare_data(self):
        datasets.FashionMNIST(self.data_dir, train=True, download=True)
        datasets.FashionMNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = FashionMNISTSuperpixelDataset(self.data_dir, train=True, transform=self.transform, download=True)
            self.val_dataset = FashionMNISTSuperpixelDataset(self.data_dir, train=False, transform=self.transform, download=True)

    def train_dataloader(self):
        return GeoDataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return GeoDataLoader(self.val_dataset, batch_size=self.batch_size)

# Training

In [93]:
dm = FashionMNISTDataModule()

dm.prepare_data()
dm.setup()

model = HGNN(num_features=1, num_classes=10)
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | gat1    | GATConv          | 1.0 K  | train
1 | gat2    | GATConv          | 66.3 K | train
2 | gat3    | GATConv          | 66.3 K | train
3 | fc      | Linear           | 2.6 K  | train
4 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.545     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Training: |                                                                                      | 0/? [00:00<…

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (82,) + inhomogeneous part.